## Creating Shapefile for Mapping
Linking crime stats w/ hexagonal geographic boundaries (to enable display of crime stats upon hover via Mapbox)

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import fiona

In [2]:
sf_crime = gpd.read_file('sf_crime_data.shp')

In [3]:
sf_crime.head(2)

,city_key,incident_c,date,offense,latitude,longitude,geometry
0,SFO,73732706372,2018-11-11,Larceny/Theft,37.779992,-122.413487,POINT (37.780 -122.413)
1,SFO,76224509035,2018-11-11,Bad Checks,37.801481,-122.416133,POINT (37.801 -122.416)


In [4]:
# Shapefile of hexagonal subsections of San Francisco (produced in QGIS)
hex_bins = gpd.read_file('sf_hexbins.shp')

In [5]:
hex_bins.head()

,left,bottom,right,top,NUMPOINTS,geometry
0,-122.512,37.770272,-122.509333,37.772582,5.0,"POLYGON ((-122.51200 37.77143, -122.51133 37.7..."
1,-122.512,37.774891,-122.509333,37.777200,9.0,"POLYGON ((-122.51200 37.77605, -122.51133 37.7..."
2,-122.510,37.755261,-122.507333,37.757571,1.0,"POLYGON ((-122.51000 37.75642, -122.50933 37.7..."
3,-122.510,37.759880,-122.507333,37.762189,1.0,"POLYGON ((-122.51000 37.76103, -122.50933 37.7..."
4,-122.510,37.762189,-122.507333,37.764499,3.0,"POLYGON ((-122.51000 37.76334, -122.50933 37.7..."


In [6]:
len(hex_bins)

815

In [7]:
# NUMPOINTS is the number of crimes recorded in each hexagon
hex_bins.NUMPOINTS.value_counts()

1.0     327
2.0     177
3.0     106
4.0      63
5.0      39
6.0      22
7.0      21
8.0      18
10.0      7
11.0      7
12.0      5
9.0       5
15.0      3
14.0      2
17.0      2
28.0      2
16.0      2
20.0      1
23.0      1
13.0      1
19.0      1
32.0      1
18.0      1
21.0      1
Name: NUMPOINTS, dtype: int64

In [8]:
hex_bins.dtypes

left          float64
bottom        float64
right         float64
top           float64
NUMPOINTS     float64
geometry     geometry
dtype: object

---

In [9]:
offenses = {0:[], 1:[], 2:[], 3:[], 4:[]}
counts = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
incidents = []

for row in hex_bins.itertuples():
    hex_left = row.left
    hex_bottom = row.bottom
    hex_right = row.right
    hex_top = row.top
    offenses_dict = {}
    
    for row2 in sf_crime.itertuples():
        lat = row2.latitude
        lon = row2.longitude
        incident = row2.incident_c
        if (lon > hex_left and lon < hex_right) and (lat > hex_bottom and lat < hex_top) and (incident not in incidents):
            incidents.append(incident)
            offense = row2.offense
            if offense in offenses_dict:
                offenses_dict[offense] += 1
            else:
                offenses_dict[offense] = 1
                
    # Storing top 5 offenses and corresponding counts in separate dicts (which will be converted to dfs and joined together)    
    top5 = sorted(offenses_dict.items(), key=lambda x: x[1], reverse=True)[:5]  # Sorting offenses dict by values in descending order
    top5_count = len(top5)
    
    for i in range(top5_count):
        offenses[i].append(top5[i][0])
        counts[i].append(top5[i][1])
    if top5_count < 5:
        for i in range(top5_count, 5):
            offenses[i].append(np.nan)
            counts[i].append(np.nan)
    counts[5].append(sum([x[1] for x in top5]))

In [10]:
off_df = pd.DataFrame.from_dict(offenses)
off_df.columns = ['Offense1', 'Offense2', 'Offense3', 'Offense4', 'Offense5']
off_df.head()

,Offense1,Offense2,Offense3,Offense4,Offense5
0,Larceny/Theft,Fraud,NaN,NaN,NaN
1,Larceny/Theft,Other,Assault,Weapons Violations,NaN
2,Assault,NaN,NaN,NaN,NaN
3,Fraud,NaN,NaN,NaN,NaN
4,Other,Drugs,Larceny/Theft,NaN,NaN


In [11]:
counts_df = pd.DataFrame.from_dict(counts)
counts_df.columns = ['Count1', 'Count2', 'Count3', 'Count4', 'Count5', 'Total_Count']
counts_df.head()

,Count1,Count2,Count3,Count4,Count5,Total_Count
0,4.0,1.0,NaN,NaN,NaN,5
1,3.0,2.0,2.0,2.0,NaN,9
2,1.0,NaN,NaN,NaN,NaN,1
3,1.0,NaN,NaN,NaN,NaN,1
4,1.0,1.0,1.0,NaN,NaN,3


In [12]:
top5_offenses = off_df.join(counts_df)
top5_offenses.head()

,Offense1,Offense2,Offense3,Offense4,Offense5,Count1,Count2,Count3,Count4,Count5,Total_Count
0,Larceny/Theft,Fraud,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,5
1,Larceny/Theft,Other,Assault,Weapons Violations,NaN,3.0,2.0,2.0,2.0,NaN,9
2,Assault,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
3,Fraud,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
4,Other,Drugs,Larceny/Theft,NaN,NaN,1.0,1.0,1.0,NaN,NaN,3


In [13]:
hex_offenses = hex_bins.join(top5_offenses)

In [14]:
hex_offenses.head(15)

,left,bottom,right,top,NUMPOINTS,geometry,Offense1,Offense2,Offense3,Offense4,Offense5,Count1,Count2,Count3,Count4,Count5,Total_Count
0,-122.512,37.770272,-122.509333,37.772582,5.0,"POLYGON ((-122.51200 37.77143, -122.51133 37.7...",Larceny/Theft,Fraud,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,5
1,-122.512,37.774891,-122.509333,37.777200,9.0,"POLYGON ((-122.51200 37.77605, -122.51133 37.7...",Larceny/Theft,Other,Assault,Weapons Violations,NaN,3.0,2.0,2.0,2.0,NaN,9
2,-122.510,37.755261,-122.507333,37.757571,1.0,"POLYGON ((-122.51000 37.75642, -122.50933 37.7...",Assault,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
3,-122.510,37.759880,-122.507333,37.762189,1.0,"POLYGON ((-122.51000 37.76103, -122.50933 37.7...",Fraud,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
4,-122.510,37.762189,-122.507333,37.764499,3.0,"POLYGON ((-122.51000 37.76334, -122.50933 37.7...",Other,Drugs,Larceny/Theft,NaN,NaN,1.0,1.0,1.0,NaN,NaN,3
5,-122.510,37.769118,-122.507333,37.771427,1.0,"POLYGON ((-122.51000 37.77027, -122.50933 37.7...",Arson,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
6,-122.510,37.776046,-122.507333,37.778355,1.0,"POLYGON ((-122.51000 37.77720, -122.50933 37.7...",Suspicious Activity,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
7,-122.508,37.733322,-122.505333,37.735631,3.0,"POLYGON ((-122.50800 37.73448, -122.50733 37.7...",Assault,Sex Offenses,Larceny/Theft,NaN,NaN,1.0,1.0,1.0,NaN,NaN,3
8,-122.508,37.751797,-122.505333,37.754106,1.0,"POLYGON ((-122.50800 37.75295, -122.50733 37.7...",Other,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
9,-122.508,37.761035,-122.505333,37.763344,2.0,"POLYGON ((-122.50800 37.76219, -122.50733 37.7...",Other,Motor Vehicle Theft,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2


In [15]:
hex_offenses[hex_offenses.NUMPOINTS > hex_offenses.Total_Count].shape[0]

101

In [16]:
hex_offenses[hex_offenses.NUMPOINTS < hex_offenses.Total_Count].shape[0]

92

In [17]:
hex_offenses.Total_Count.sum()

2425

In [18]:
hex_offenses.NUMPOINTS.sum()

2462.0

In [19]:
len(sf_crime)

2464

Resolving Overlap Issues:
- Hex bin coordinates represent rectangles (top, bottom, left, right), resulting in about 20% of data being represented twice (this produces overlap in rectangle corners, resulting in double counts for offenses whose coordinates fall in the overlapping areas)
- I only included the first instance of an offense to prevent double counting (this results in much more accurate total offense count in each hex, but might categorize offense in wrong hex, which isn't really a big deal since they're adjascent)

Next Step:
- Export to shapefile, import in mapbox html file & add interactivity via Mapbox GL JS

#### Converting to Shapefile:

In [20]:
# Contains crime data for each hexbin (will map and add interactivity via Mapbox)
hex_offenses.to_file('hex_offenses.shp')